In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import matplotlib.pyplot as plt
import pydot
import graphviz
import seaborn as sns
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Add, Activation, Dense, SeparableConv2D, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import cv2

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
def get_data(data_dir):
    img_dim = 256
    labels = ["NORMAL","PNEUMONIA"]
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        for file in os.listdir(path):
            img = cv2.imread(os.path.join(path, file), cv2.IMREAD_GRAYSCALE)
            img_resize = cv2.resize(img, (img_dim, img_dim), interpolation=cv2.INTER_CUBIC)
            data.append([np.array(img_resize), labels.index(label)])

    return np.array(data, dtype=object)
def preprocess(data):
    #must be in form [(x,y),...]
    x_data = []
    y_data = []
    for x, y in data:
        x_data.append(x)
        y_data.append(y)
    return np.expand_dims(np.array(x_data), axis=-1), np.expand_dims(np.array(y_data), axis=-1)

In [ ]:
train = get_data("./chest_xray/train")
test = get_data("./chest_xray/test") 
val = get_data("./chest_xray/val")

In [ ]:
x_train, y_train = preprocess(train)
x_test, y_test = preprocess(test)
x_val, y_val = preprocess(val)

In [ ]:
#train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
#val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
train_dataset = tf.keras.utils.image_dataset_from_directory("./chest_xray/train",
                                                            color_mode = 'grayscale',
                                                            batch_size = 8,
                                                            interpolation = 'bicubic'
                                                           )

test_dataset = tf.keras.utils.image_dataset_from_directory("./chest_xray/test",
                                                            color_mode = 'grayscale',
                                                            batch_size = 8,
                                                            interpolation = 'bicubic'
                                                           )
val_dataset = tf.keras.utils.image_dataset_from_directory("./chest_xray/val",
                                                            color_mode = 'grayscale',
                                                            batch_size = 8,
                                                            interpolation = 'bicubic'
                                                           )

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].imshow(x_val[0], cmap="gray")
ax[0].title.set_text("Normal")
ax[1].imshow(x_val[-1], cmap="gray")
ax[1].title.set_text("Pneumonia")

In [ ]:
count_0 = 0
count_1 = 0
for x in range(len(y_train)):
    if y_train[x] == 0:
        count_0 += 1
    else:
        count_1 += 1

fig, ax = plt.subplots()
bar = ax.bar(["NORMAL", "PNEUMONIA"], [count_0, count_1])
ax.bar_label(bar)
plt.title("Data Split")
        

In [ ]:
def cnn_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)
    
    x = Conv2D(128, 3, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    previous = x #residual to improve neuron independence
    
    for size in [256, 512, 728]:
        x = Activation("relu")(x)
        x = SeparableConv2D(size, 3, padding="same")(x)
        x = BatchNormalization()(x)
        
        x = Activation("relu")(x)
        x = SeparableConv2D(size, 3, padding="same")(x)
        x = BatchNormalization()(x)
        
        x = keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)
        
        residual = Conv2D(size, 1, strides=2, padding="same")(previous)
        x = keras.layers.add([x, residual])
        previous = x
        
    x = SeparableConv2D(1024, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    
    return keras.Model(inputs, outputs)
    

In [ ]:
model = cnn_model((256,256,1))


In [ ]:
model.summary()
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
num_epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(
    train_dataset,
    epochs =  num_epochs,
    validation_data = val_dataset,
    callbacks = callbacks
)

In [ ]:
test_results = model.evaluate(test_dataset)
print("Loss of the model is - " , test_results[0])
print("Accuracy of the model is - " , test_results[1]*100 , "%")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
results = []
for i in range(1,25):
    testmodel = keras.models.load_model("save_at_" + str(i) + ".keras")
    results.append(testmodel.evaluate(test_dataset))

plt.plot(results)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()


In [ ]:
plt.plot(results)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()


In [ ]:
testmodel = keras.models.load_model("save_at_13.keras")

In [ ]:
testmodel.evaluate(test_dataset)

In [ ]:
testmodel 

In [ ]:
filters, bias = testmodel.layers[1].get_weights()
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

n_filters, ix = 6, 1
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # specify subplot and turn of axis
    ax = plt.subplot(n_filters, 6, ix)
    ax.set_xticks([])
    ax.set_yticks([])
    # plot filter channel in grayscale
    plt.imshow(f[:, :], cmap='gray')
    ix += 1
# show the figure
plt.show()

In [ ]:
filtermodel = keras.Model(inputs=testmodel.inputs, outputs=testmodel.layers[1].output)
filtermodel.summary()

In [ ]:
maps = filtermodel.predict(x_val)

In [ ]:
filter_img = maps[0,:,:,:]

In [ ]:
filter_img.shape

In [ ]:
filter_img = maps[0,:,:,:]
ix = 1
for _ in range(8):
    for _ in range(16):
    # specify subplot and turn of axis
        ax = plt.subplot(8, 16, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(filter_img[:, :, ix-1], cmap='gray')
        ix += 1
# show the figure
plt.show()

In [ ]:
filter_img = maps[-1,:,:,:]
ix = 1
for _ in range(8):
    for _ in range(16):
    # specify subplot and turn of axis
        ax = plt.subplot(8, 16, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(filter_img[:, :, ix-1], cmap='gray')
        ix += 1
# show the figure
plt.show()